In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

sns.set_style("whitegrid")

In [ ]:
import pandas as pd

# Load stock data
df = pd.read_csv("../Datas/yfinance_data/Data/AAPL.csv", parse_dates=['Date'])
df.set_index('Date', inplace=True)

# -----------------------------
# 1. Simple Moving Average (SMA)
# -----------------------------
df['SMA_20'] = df['Close'].rolling(window=20).mean()

# -----------------------------
# 2. Exponential Moving Average (EMA)
# -----------------------------
df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()

# -----------------------------
# 3. Relative Strength Index (RSI)
# -----------------------------
delta = df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# -----------------------------
# 4. MACD
# -----------------------------
ema12 = df['Close'].ewm(span=12, adjust=False).mean()
ema26 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = ema12 - ema26
df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['MACD_hist'] = df['MACD'] - df['MACD_signal']

# -----------------------------
# Save processed CSV
# -----------------------------
df.to_csv("../Datas/processed/AAPL_processed.csv")

print("Indicators calculated and CSV saved.")


In [ ]:
# Folders
data_folder = "../Datas/yfinance_data/Data/"
processed_folder = "../Datas/processed/"
os.makedirs(processed_folder, exist_ok=True)

# List all stock CSVs
stock_files = glob.glob(data_folder + "*.csv")
print("Stocks found:", [os.path.basename(f) for f in stock_files])


In [ ]:
def load_stock(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df.set_index('Date', inplace=True)
    return df

def add_technical_indicators(df):
    # Simple Moving Average (SMA)
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    
    # Exponential Moving Average (EMA)
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    
    # Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    
    return df


In [ ]:
for stock_file in stock_files:
    ticker = os.path.basename(stock_file).split('.')[0]
    
    # Load stock
    df = load_stock(stock_file)
    
    # Add indicators
    df = add_technical_indicators(df)
    
    # Save processed CSV
    df.to_csv(f"{processed_folder}{ticker}_processed.csv")
    print(f"{ticker} processed and saved.")


In [ ]:
df = pd.read_csv(f"{processed_folder}AAPL_processed.csv", parse_dates=['Date'])
df.set_index('Date', inplace=True)

plt.figure(figsize=(14,7))
plt.plot(df.index, df['Close'], label='Close Price', color='blue')
plt.plot(df.index, df['SMA_20'], label='SMA 20', color='orange')
plt.plot(df.index, df['EMA_20'], label='EMA 20', color='green')
plt.title('AAPL Stock Price with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# RSI plot
plt.figure(figsize=(12,4))
plt.plot(df.index, df['RSI'], label='RSI', color='purple')
plt.axhline(70, color='red', linestyle='--')
plt.axhline(30, color='green', linestyle='--')
plt.title('Relative Strength Index (RSI)')
plt.show()
